In [19]:
import pandas 

new_dedup_data = pandas.read_csv('./stats.csv')
data = pandas.read_csv('./bigcode_pls.csv')

In [29]:
data = data[:-1]
included_pls = data[data['Include'] == 1]['lang']
included_data = new_dedup_data[new_dedup_data['table_id'].isin(included_pls)]
included_data = included_data.rename(columns={'table_id': 'lang'})

In [46]:
import plotly.express as px
fig = px.bar(included_data.sort_values("size_after_gb"), x="lang", y="size_after_gb", labels={'size_after_gb': 'Size (GB)'})
fig.show()

In [45]:
notebooks_row = pandas.DataFrame({'lang':'Notebooks', 'size_after_gb': 20}, index =[0])
issues_row = pandas.DataFrame({'lang':'GH issues', 'size_after_gb': 100}, index =[0])
commits_row = pandas.DataFrame({'lang':'Git commits', 'size_after_gb': 80}, index =[0])

concat_data = pandas.concat([notebooks_row, issues_row, commits_row, included_data])
concat_data = concat_data.sort_values('size_after_gb')
fig = px.bar(concat_data, x="lang", y="size_after_gb", labels={'size_after_gb': 'Size (GB)'})
fig.update_layout(yaxis_range=[0,180])
fig.show()

In [55]:
def unimax(data, N, token_budget):
    B = token_budget
    i = 0
    tokens = list()
    
    # assume data sorted, size in GB
    for _, l in data.iterrows():
        row = {'lang': l['lang'], 'tokens': 0}
        b_l = B / (len(data) - i)
        u = l['size_after_gb']/char_to_tokens
        if b_l > N*u:
            row['tokens'] = N*u
        else:
            row['tokens']= b_l
        B = B - row['tokens']
        i += 1
        tokens.append(row)
    return tokens, B

num_tokens = 800
char_to_tokens = 4
unimax_N = 4

lang2tokens, budget_left = unimax(concat_data, unimax_N, num_tokens)
if budget_left > 0.0:
    print(f"""Still {budget_left} tokens left for training!""")
else:    
    tokens_df = pandas.DataFrame(lang2tokens)

    fig = px.bar(tokens_df, x="lang", y="tokens", labels={'tokens': 'Tokens (B)'})
    fig.show()